# Acervo Estadão - Greves <a class="tocSkip">

<font size=4>Danilo Paula de Souza</font> <br>
Ph.D candidate in Economics | University of Sao Paulo <br>
[https://sites.google.com/view/dpsouza](https://sites.google.com/view/dpsouza) | [danilo.paulasouza@hotmail.com](mailto:danilo.paulasouza@hotmail.com)<br><br>

Version: July 18th, 2018

***

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages-and-functions" data-toc-modified-id="Packages-and-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages and functions</a></span></li><li><span><a href="#Inputs-da-análise" data-toc-modified-id="Inputs-da-análise-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Inputs da análise</a></span></li><li><span><a href="#Webscraping-das-notícias" data-toc-modified-id="Webscraping-das-notícias-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Webscraping das notícias</a></span><ul class="toc-item"><li><span><a href="#Preferências-do-webdriver" data-toc-modified-id="Preferências-do-webdriver-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Preferências do webdriver</a></span></li><li><span><a href="#Extraindo-os-links-das-páginas-das-imagens" data-toc-modified-id="Extraindo-os-links-das-páginas-das-imagens-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Extraindo os links das páginas das imagens</a></span></li><li><span><a href="#Download-do-arquivo-html-de-cada-notícia-individual" data-toc-modified-id="Download-do-arquivo-html-de-cada-notícia-individual-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Download do arquivo html de cada notícia individual</a></span></li></ul></li><li><span><a href="#Compilando-as-informações-das-notícias" data-toc-modified-id="Compilando-as-informações-das-notícias-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compilando as informações das notícias</a></span><ul class="toc-item"><li><span><a href="#Extraindo-as-informações-de-cada-notícia-individual" data-toc-modified-id="Extraindo-as-informações-de-cada-notícia-individual-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Extraindo as informações de cada notícia individual</a></span></li><li><span><a href="#Salvando-os-resultados-em-um-csv" data-toc-modified-id="Salvando-os-resultados-em-um-csv-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Salvando os resultados em um csv</a></span></li><li><span><a href="#Mantendo-apenas-notícias-de-greve-de-bancários" data-toc-modified-id="Mantendo-apenas-notícias-de-greve-de-bancários-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Mantendo apenas notícias de greve de bancários</a></span></li><li><span><a href="#Definindo-a-categoria-da-greve" data-toc-modified-id="Definindo-a-categoria-da-greve-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Definindo a categoria da greve</a></span></li><li><span><a href="#Definindo-a-duração-da-greve" data-toc-modified-id="Definindo-a-duração-da-greve-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Definindo a duração da greve</a></span></li><li><span><a href="#Definindo-o-local-da-greve" data-toc-modified-id="Definindo-o-local-da-greve-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Definindo o local da greve</a></span></li></ul></li></ul></div>

## Packages and functions

In [1]:
import os
import re
import sys
import time
import calendar
import requests
import numpy as np
import pandas as pd
import nltk 
import pprint
import shutil

from selenium import webdriver
from bs4 import BeautifulSoup as BSoup
from numpy.random import uniform
from scrapy.selector import Selector
from unidecode import unidecode

from selenium.webdriver.common.keys import Keys

def get_random_ua():
    random_ua = ''
    ua_file = 'ua_file.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
            lines = [re.sub('\n$','',elem) for elem in lines]
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

def get_random_ip():
    random_ip = ''
    ips_file = 'ips_file.txt'
    try:
        with open(ips_file) as f:
            lines = f.readlines()
            lines = [re.sub('\n$','',elem) for elem in lines]
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ip = lines[int(idx)]
    except Exception as ex:
        print('Exception in random_ip')
        print(str(ex))
    finally:
        return random_ip

## Inputs da análise

In [2]:
words = 'greve'
words_to_archive = words
Y0 = 2000
YT = 2018
M0 = 1
MT = 6

months = list(range(1,12+1))
years  = list(range(Y0,YT+1))

pc = 'pessoal'

if pc == 'pessoal':
    os.chdir('D:/Dropbox/Projetos - Corbi/Acervo Estadao/html e csv')
elif pc == 'fipe':
    os.chdir('C:/Users/danilo.souza/Desktop/Acervo Estadao/html e csv')

path_chrome = 'C:/Users/danilo.souza/Desktop/Acervo Estadao/python_selenium_drivers/chromedriver_v2_38.exe'

## Webscraping das notícias

### Preferências do webdriver

In [4]:
options = webdriver.ChromeOptions()
prefs = {'profile.managed_default_content_settings.images':2,'profile.default_content_setting_values.notifications': 2,'disk-cache-size': 4096}
options.add_experimental_option('prefs',prefs)

### Extraindo os links das páginas das imagens

In [5]:
browser = webdriver.Chrome(executable_path = path_chrome, chrome_options=options)
browser.get('http://busca.estadao.com.br/')
url_base = 'http://busca.estadao.com.br/?tipo_conteudo=Not%C3%ADcias&quando=_D0_%2F_M0_%2F_Y0_-_DT_%2F_MT_%2F_YT_&q='+words

def search_btn_carregar_mais():
    try:
        browser.find_element_by_link_text('CARREGAR MAIS')
        return True
    except:
        return False   

lista_erros = []
for y in years:
    for m in months:
        if (y+m/10) >= (Y0+M0/10) and (y+m/10) <= (YT+MT/10):
            for d in range(1,calendar.monthrange(y,m)[1]):
            
                url_actual = url_base.replace('_D0_',str(d).zfill(2))
                url_actual = url_actual.replace('_M0_',str(m).zfill(2))
                url_actual = url_actual.replace('_Y0_',str(y))
                url_actual = url_actual.replace('_DT_',str(d).zfill(2))
                url_actual = url_actual.replace('_MT_',str(m).zfill(2))
                url_actual = url_actual.replace('_YT_',str(y))

                browser.get(url_actual)
                browser.refresh() 

                while search_btn_carregar_mais():
                    browser.find_element_by_link_text('CARREGAR MAIS').send_keys('\n')
                    time.sleep(1 + uniform())
                
                bs_obj = BSoup(browser.page_source, 'lxml')
                noticias = bs_obj.find_all('a', class_='link-title')
                
                num_registros = int(re.search('\d+',bs_obj.find('p', class_='flt-registros').text).group(0))
                if len(noticias) != num_registros:
                    print('DEU ERRO NAS NOTÍCIAS DESSE DIA: '+str(d).zfill(2)+'/'+str(m).zfill(2)+'/'+str(y))
                    lista_erros.append(str(d).zfill(2)+'/'+str(m).zfill(2)+'/'+str(y))
                
                links = []
                for noticia in noticias:
                    link = noticia['href']
                    link = re.search('(.*)(estadao\.com\.br/noticias)(.*)', link)
                    if link != None:
                        links.append(link.group(0))
                        
                links = [elem+'___'+str(y)+str(m).zfill(2)+str(d).zfill(2)+'_'+str(ind+1).zfill(3) for ind,elem in enumerate(links)]
                        
                urls_news = open('urls_news_'+words_to_archive+'_'+str(y)+str(m).zfill(2)+str(d).zfill(2)+'.csv',mode='w',encoding='utf8')
                urls_news.write('\n'.join(links))
                urls_news.close()

DEU ERRO NAS NOTÍCIAS DESSE DIA: 24/05/2018
DEU ERRO NAS NOTÍCIAS DESSE DIA: 27/05/2018
DEU ERRO NAS NOTÍCIAS DESSE DIA: 14/06/2018


Por algum motivo X pode dar erro no código nessas notícias que aparecem na lista de erros. O código auxiliar abaixo serve para rodar novamente apenas as páginas que deram erro e estão na lista de erros. Note que se der erro de novo em algo que já tinha dado o código vai readicionar esse erro à lista e vai tentar até não haver mais erro nenhum.

In [46]:
url_base = 'http://busca.estadao.com.br/?tipo_conteudo=Not%C3%ADcias&quando=_D0_%2F_M0_%2F_Y0_-_DT_%2F_MT_%2F_YT_&q='+words

def search_btn_carregar_mais():
    try:
        browser.find_element_by_link_text('CARREGAR MAIS')
        return True
    except:
        return False

for erro in lista_erros:
    
    d = erro[0:2]
    m = erro[3:5]
    y = erro[6:]
    
    url_actual = url_base.replace('_D0_',str(d).zfill(2))
    url_actual = url_actual.replace('_M0_',str(m).zfill(2))
    url_actual = url_actual.replace('_Y0_',str(y))
    url_actual = url_actual.replace('_DT_',str(d).zfill(2))
    url_actual = url_actual.replace('_MT_',str(m).zfill(2))
    url_actual = url_actual.replace('_YT_',str(y))

    browser.get(url_actual)
    browser.refresh() 

    while search_btn_carregar_mais():
        browser.find_element_by_link_text('CARREGAR MAIS').send_keys('\n')
        time.sleep(1 + uniform())
                
    bs_obj = BSoup(browser.page_source, 'lxml')
    noticias = bs_obj.find_all('a', class_='link-title')
                
    num_registros = int(re.search('\d+',bs_obj.find('p', class_='flt-registros').text).group(0))
    if len(noticias) != num_registros:
        print('DEU ERRO NAS NOTÍCIAS DESSE DIA: '+str(d).zfill(2)+'/'+str(m).zfill(2)+'/'+str(y))
        lista_erros.append(str(d).zfill(2)+'/'+str(m).zfill(2)+'/'+str(y))
                
    links = []
    for noticia in noticias:
        link = noticia['href']
        link = re.search('(.*)(estadao\.com\.br/noticias)(.*)', link)
        if link != None:
            links.append(link.group(0))
                        
    links = [elem+'___'+str(y)+str(m).zfill(2)+str(d).zfill(2)+'_'+str(ind+1).zfill(3) for ind,elem in enumerate(links)]
                        
    urls_news = open('urls_news_'+words_to_archive+'_'+str(y)+str(m).zfill(2)+str(d).zfill(2)+'.csv',mode='w',encoding='utf8')
    urls_news.write('\n'.join(links))
    urls_news.close()

DEU ERRO NAS NOTÍCIAS DESSE DIA: 01/02/2011


In [19]:
files = [f for f in os.listdir('.') if re.match(r'[a-z0-9_]+.csv', f)]

links = []

for file in files:
    if os.path.getsize(file)>0:
        link_aux = pd.read_csv(file,header=None,sep=' ')[0].values.tolist()
        links = links + link_aux

categoria = [re.findall('https://(.*)\.estadao',elem)[0] if re.findall('https://(.*)\.estadao',elem) != [] else 'nan'  for elem in links]
caderno   = [re.findall('noticias/(.*)',elem)[0].split(',')[0] for elem in links]

df_news = pd.DataFrame({'caderno':caderno,'categoria':categoria,'links':links})
df_news.head()

,caderno,categoria,links
0,geral,economia,https://economia.estadao.com.br/noticias/geral...
1,geral,economia,https://economia.estadao.com.br/noticias/geral...
2,geral,cultura,"https://cultura.estadao.com.br/noticias/geral,..."
3,geral,economia,https://economia.estadao.com.br/noticias/geral...
4,geral,economia,https://economia.estadao.com.br/noticias/geral...


### Download do arquivo html de cada notícia individual

In [27]:
files = []

links = links2[23284:]

iterations = len(links)
for it, link in enumerate(links):
    
    file_name = re.findall('___(.*)',link)[0]+'.html'
    link = re.findall('(.*)___',link)[0]
    
    # if os.path.isfile(file_name):
       # print('\rHtml number ' + str(it+1) + ' has already been downloaded.' )
    
    if not os.path.isfile(file_name): # baixa apenas as imagens que não existem na pasta
        
        user_agent = get_random_ua()
        proxy_url = 'https://'+get_random_ip()
        headers = {'User-Agent': user_agent}
        time.sleep(1 + uniform())
        
        print('\rFetching html number ' + str(it+1) + '/' + str(iterations)+'. There are '+str(iterations-it-1)+' files left.' )
        
        
        
        try:
            r = requests.get(link, headers=headers, proxies={'https': proxy_url})
            print('\rProxy IP working: '+str(proxy_url))
        except:
            r = requests.get(link, headers=headers)
            
            
        with open(file_name, 'wb') as file:
            file.write(r.content)
        print('\rDownload complete! \n')
        
        files.append(file_name)

Fetching html number 2633/2713. There are 80 files left.
Download complete! 

Fetching html number 2634/2713. There are 79 files left.
Download complete! 

Fetching html number 2635/2713. There are 78 files left.
Download complete! 

Fetching html number 2636/2713. There are 77 files left.
Download complete! 

Fetching html number 2637/2713. There are 76 files left.
Proxy IP working: https://93.179.219.71:41258
Download complete! 

Fetching html number 2638/2713. There are 75 files left.
Proxy IP working: https://24.92.171.194:8080
Download complete! 

Fetching html number 2639/2713. There are 74 files left.
Download complete! 

Fetching html number 2640/2713. There are 73 files left.
Proxy IP working: https://140.227.8.115:3129
Download complete! 

Fetching html number 2641/2713. There are 72 files left.
Proxy IP working: https://91.250.3.101:53281
Download complete! 

Fetching html number 2642/2713. There are 71 files left.
Download complete! 

Fetching html number 2643/2713. There a

## Compilando as informações das notícias

### Extraindo as informações de cada notícia individual

In [63]:
files_csv  = [f for f in os.listdir('.') if re.match(r'[a-z0-9_]+.csv' , f)][1:]
files_html = [f for f in os.listdir('.') if re.match(r'[0-9\_]+.html', f)]

links = []

for file in files_csv:
    if os.path.getsize(file)>0:
        link_aux = pd.read_csv(file,header=None,sep=' ')[0].values.tolist()
        links = links + link_aux

categoria = [re.findall('https://(.*)\.estadao',elem)[0] if re.findall('https://(.*)\.estadao',elem) != [] else 'nan'  for elem in links]
caderno   = [re.findall('noticias/(.*)',elem)[0].split(',')[0] for elem in links]

df_news = pd.DataFrame({'caderno':caderno,'categoria':categoria,'links':links})
df_news['files'] = files_html

df_news.describe()

,caderno,categoria,links,files
count,25997,25997,25997,25997
unique,47,21,25997,25997
top,geral,economia,https://educacao.estadao.com.br/noticias/geral...,20140606_004.html
freq,22141,5071,1,1


In [65]:
manchetes = []
subtitulo = []
autor     = []
conteudo  = []
erros     = []

iterations = len(df_news['files'])
for it, file in enumerate(df_news['files']):
    
    html_data = open(file, mode='r',encoding='latin1').read()
    
    manchete_atual  = Selector(text=html_data).xpath('//*[@class="n--noticia__title"]/text()').extract_first()
    autor_atual     = Selector(text=html_data).xpath('//*[@class="n--noticia__state"]//text()').extract()
    autor_atual = ' '.join(autor_atual)
    autor_atual = re.sub('\\s+',' ',autor_atual)
    autor_atual = re.sub('^\s|\s$','',autor_atual)
    
    if len(Selector(text=html_data).xpath('//*[contains(@class,"n--noticia__content")]/p/text()')) != 0:
        subtitulo_atual = Selector(text=html_data).xpath('//*[@class="n--noticia__subtitle"]/text()').extract_first()
        conteudo_atual  = Selector(text=html_data).xpath('//*[contains(@class,"n--noticia__content")]//text()').extract()
              
    else:
        subtitulo_atual = ''
        conteudo_atual  = Selector(text=html_data).xpath('//*[@class="n--noticia__subtitle"]/text()').extract()
    
    
    conteudo_atual  = ' '.join(conteudo_atual)
    
    try: 
        manchete_atual = re.sub('\\s+',' ',manchete_atual)
        manchete_atual = re.sub('^\s|\s$','',manchete_atual)
        subtitulo_atual = re.sub('\\s+',' ',subtitulo_atual)
        subtitulo_atual = re.sub('^\s|\s$','',subtitulo_atual)
        autor_atual = re.sub('\\s+',' ',autor_atual)
        autor_atual = re.sub('^\s|\s$','',autor_atual)
        conteudo_atual = re.sub('\\s+',' ',conteudo_atual)
        conteudo_atual = re.sub('^\s|\s$','',conteudo_atual)
    except:
        manchete_atual = ''
        subtitulo_atual = ''
        autor_atual = ''
        conteudo_atual = ''
        erros.append(file)
        
    manchetes.append(manchete_atual.lower())
    subtitulo.append(subtitulo_atual)
    autor.append(autor_atual)
    conteudo.append(conteudo_atual)
    
    print('Reading html number ' + str(it+1).zfill(5) + '/' + str(iterations))

df_news['manchete']  = manchetes
df_news['subtitulo'] = subtitulo
df_news['autor']     = autor
df_news['conteudo']  = conteudo

Reading html number 00001/25997
Reading html number 00002/25997
Reading html number 00003/25997
Reading html number 00004/25997
Reading html number 00005/25997
Reading html number 00006/25997
Reading html number 00007/25997
Reading html number 00008/25997
Reading html number 00009/25997
Reading html number 00010/25997
Reading html number 00011/25997
Reading html number 00012/25997
Reading html number 00013/25997
Reading html number 00014/25997
Reading html number 00015/25997
Reading html number 00016/25997
Reading html number 00017/25997
Reading html number 00018/25997
Reading html number 00019/25997
Reading html number 00020/25997
Reading html number 00021/25997
Reading html number 00022/25997
Reading html number 00023/25997
Reading html number 00024/25997
Reading html number 00025/25997
Reading html number 00026/25997
Reading html number 00027/25997
Reading html number 00028/25997
Reading html number 00029/25997
Reading html number 00030/25997
Reading html number 00031/25997
Reading 

Reading html number 00259/25997
Reading html number 00260/25997
Reading html number 00261/25997
Reading html number 00262/25997
Reading html number 00263/25997
Reading html number 00264/25997
Reading html number 00265/25997
Reading html number 00266/25997
Reading html number 00267/25997
Reading html number 00268/25997
Reading html number 00269/25997
Reading html number 00270/25997
Reading html number 00271/25997
Reading html number 00272/25997
Reading html number 00273/25997
Reading html number 00274/25997
Reading html number 00275/25997
Reading html number 00276/25997
Reading html number 00277/25997
Reading html number 00278/25997
Reading html number 00279/25997
Reading html number 00280/25997
Reading html number 00281/25997
Reading html number 00282/25997
Reading html number 00283/25997
Reading html number 00284/25997
Reading html number 00285/25997
Reading html number 00286/25997
Reading html number 00287/25997
Reading html number 00288/25997
Reading html number 00289/25997
Reading 

Reading html number 00518/25997
Reading html number 00519/25997
Reading html number 00520/25997
Reading html number 00521/25997
Reading html number 00522/25997
Reading html number 00523/25997
Reading html number 00524/25997
Reading html number 00525/25997
Reading html number 00526/25997
Reading html number 00527/25997
Reading html number 00528/25997
Reading html number 00529/25997
Reading html number 00530/25997
Reading html number 00531/25997
Reading html number 00532/25997
Reading html number 00533/25997
Reading html number 00534/25997
Reading html number 00535/25997
Reading html number 00536/25997
Reading html number 00537/25997
Reading html number 00538/25997
Reading html number 00539/25997
Reading html number 00540/25997
Reading html number 00541/25997
Reading html number 00542/25997
Reading html number 00543/25997
Reading html number 00544/25997
Reading html number 00545/25997
Reading html number 00546/25997
Reading html number 00547/25997
Reading html number 00548/25997
Reading 

Reading html number 00776/25997
Reading html number 00777/25997
Reading html number 00778/25997
Reading html number 00779/25997
Reading html number 00780/25997
Reading html number 00781/25997
Reading html number 00782/25997
Reading html number 00783/25997
Reading html number 00784/25997
Reading html number 00785/25997
Reading html number 00786/25997
Reading html number 00787/25997
Reading html number 00788/25997
Reading html number 00789/25997
Reading html number 00790/25997
Reading html number 00791/25997
Reading html number 00792/25997
Reading html number 00793/25997
Reading html number 00794/25997
Reading html number 00795/25997
Reading html number 00796/25997
Reading html number 00797/25997
Reading html number 00798/25997
Reading html number 00799/25997
Reading html number 00800/25997
Reading html number 00801/25997
Reading html number 00802/25997
Reading html number 00803/25997
Reading html number 00804/25997
Reading html number 00805/25997
Reading html number 00806/25997
Reading 

Reading html number 01033/25997
Reading html number 01034/25997
Reading html number 01035/25997
Reading html number 01036/25997
Reading html number 01037/25997
Reading html number 01038/25997
Reading html number 01039/25997
Reading html number 01040/25997
Reading html number 01041/25997
Reading html number 01042/25997
Reading html number 01043/25997
Reading html number 01044/25997
Reading html number 01045/25997
Reading html number 01046/25997
Reading html number 01047/25997
Reading html number 01048/25997
Reading html number 01049/25997
Reading html number 01050/25997
Reading html number 01051/25997
Reading html number 01052/25997
Reading html number 01053/25997
Reading html number 01054/25997
Reading html number 01055/25997
Reading html number 01056/25997
Reading html number 01057/25997
Reading html number 01058/25997
Reading html number 01059/25997
Reading html number 01060/25997
Reading html number 01061/25997
Reading html number 01062/25997
Reading html number 01063/25997
Reading 

...

Reading html number 24473/25997
Reading html number 24474/25997
Reading html number 24475/25997
Reading html number 24476/25997
Reading html number 24477/25997
Reading html number 24478/25997
Reading html number 24479/25997
Reading html number 24480/25997
Reading html number 24481/25997
Reading html number 24482/25997
Reading html number 24483/25997
Reading html number 24484/25997
Reading html number 24485/25997
Reading html number 24486/25997
Reading html number 24487/25997
Reading html number 24488/25997
Reading html number 24489/25997
Reading html number 24490/25997
Reading html number 24491/25997
Reading html number 24492/25997
Reading html number 24493/25997
Reading html number 24494/25997
Reading html number 24495/25997
Reading html number 24496/25997
Reading html number 24497/25997
Reading html number 24498/25997
Reading html number 24499/25997
Reading html number 24500/25997
Reading html number 24501/25997
Reading html number 24502/25997
Reading html number 24503/25997
Reading 

Reading html number 24731/25997
Reading html number 24732/25997
Reading html number 24733/25997
Reading html number 24734/25997
Reading html number 24735/25997
Reading html number 24736/25997
Reading html number 24737/25997
Reading html number 24738/25997
Reading html number 24739/25997
Reading html number 24740/25997
Reading html number 24741/25997
Reading html number 24742/25997
Reading html number 24743/25997
Reading html number 24744/25997
Reading html number 24745/25997
Reading html number 24746/25997
Reading html number 24747/25997
Reading html number 24748/25997
Reading html number 24749/25997
Reading html number 24750/25997
Reading html number 24751/25997
Reading html number 24752/25997
Reading html number 24753/25997
Reading html number 24754/25997
Reading html number 24755/25997
Reading html number 24756/25997
Reading html number 24757/25997
Reading html number 24758/25997
Reading html number 24759/25997
Reading html number 24760/25997
Reading html number 24761/25997
Reading 

Reading html number 25244/25997
Reading html number 25245/25997
Reading html number 25246/25997
Reading html number 25247/25997
Reading html number 25248/25997
Reading html number 25249/25997
Reading html number 25250/25997
Reading html number 25251/25997
Reading html number 25252/25997
Reading html number 25253/25997
Reading html number 25254/25997
Reading html number 25255/25997
Reading html number 25256/25997
Reading html number 25257/25997
Reading html number 25258/25997
Reading html number 25259/25997
Reading html number 25260/25997
Reading html number 25261/25997
Reading html number 25262/25997
Reading html number 25263/25997
Reading html number 25264/25997
Reading html number 25265/25997
Reading html number 25266/25997
Reading html number 25267/25997
Reading html number 25268/25997
Reading html number 25269/25997
Reading html number 25270/25997
Reading html number 25271/25997
Reading html number 25272/25997
Reading html number 25273/25997
Reading html number 25274/25997
Reading 

Reading html number 25503/25997
Reading html number 25504/25997
Reading html number 25505/25997
Reading html number 25506/25997
Reading html number 25507/25997
Reading html number 25508/25997
Reading html number 25509/25997
Reading html number 25510/25997
Reading html number 25511/25997
Reading html number 25512/25997
Reading html number 25513/25997
Reading html number 25514/25997
Reading html number 25515/25997
Reading html number 25516/25997
Reading html number 25517/25997
Reading html number 25518/25997
Reading html number 25519/25997
Reading html number 25520/25997
Reading html number 25521/25997
Reading html number 25522/25997
Reading html number 25523/25997
Reading html number 25524/25997
Reading html number 25525/25997
Reading html number 25526/25997
Reading html number 25527/25997
Reading html number 25528/25997
Reading html number 25529/25997
Reading html number 25530/25997
Reading html number 25531/25997
Reading html number 25532/25997
Reading html number 25533/25997
Reading 

Reading html number 25761/25997
Reading html number 25762/25997
Reading html number 25763/25997
Reading html number 25764/25997
Reading html number 25765/25997
Reading html number 25766/25997
Reading html number 25767/25997
Reading html number 25768/25997
Reading html number 25769/25997
Reading html number 25770/25997
Reading html number 25771/25997
Reading html number 25772/25997
Reading html number 25773/25997
Reading html number 25774/25997
Reading html number 25775/25997
Reading html number 25776/25997
Reading html number 25777/25997
Reading html number 25778/25997
Reading html number 25779/25997
Reading html number 25780/25997
Reading html number 25781/25997
Reading html number 25782/25997
Reading html number 25783/25997
Reading html number 25784/25997
Reading html number 25785/25997
Reading html number 25786/25997
Reading html number 25787/25997
Reading html number 25788/25997
Reading html number 25789/25997
Reading html number 25790/25997
Reading html number 25791/25997
Reading 

### Salvando os resultados em um csv

In [66]:
erros_html = open('erros_html_'+words_to_archive+'.csv',mode='w',encoding='utf8')
erros_html.write('\n'.join(erros))
erros_html.close()

df_news['links'] = [re.sub('___[0-9\_]+','',link) for link in df_news['links']]

writer = pd.ExcelWriter('df_news.xlsx')
df_news.to_excel(writer, 'raw_data')
writer.save()

C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'https://alias.estadao.com.br/noticias/geral,a-guerra-do-fim-do-mundo-se-voce-se-preocupa-com-criancas-saude-pobreza-agricultores-comida-fome-pense-em-2013-como-o-ano-zero-da-luta-contra-as-mudancas-climaticas-luta-na-qual-so-vale-ganhar-tudo-ou-tudo-perder-imp-,978979' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'https://economia.estadao.com.br/noticias/geral,oposicao-rejeita-alta-adicional-de-impostos-grupo-air-france-klm-planeja-comprar-alitalia-de-investidores-diz-jornal-governo-prepara-pacote-de-us-113-bilhoes-anglo-american-quer-tirar-ceo-de-sua-rival-deficit-fiscal-chines-crescera-em-2013-imp-,981347' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
C:\Anaconda3\lib\site-packages\xlsxwrite

### Mantendo apenas notícias de greve de bancários

In [22]:
df_greve_bancos

,caderno,categoria,links,files,manchete,subtitulo,autor,conteudo,greve,bancos
0,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",banespa: saiba o que fazer durante a greve,NaN,"Agencia Estado, 08 Novembro 2000 | 13h08",A greve de funcionários do Banespa iniciada no...,1,1
1,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",banespa: como proceder durante a greve,NaN,"Agencia Estado, 08 Novembro 2000 | 07h14",Os funcionários públicos que recebem salários ...,1,1
2,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",termina greve do banespa,NaN,"Agencia Estado, 09 Novembro 2000 | 19h03","Em assembléia realizada há pouco, foi aprovado...",1,1
3,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",banespa: assembléia para o fim da greve,NaN,"Agencia Estado, 09 Novembro 2000 | 17h14",Os cerca de 6 mil manifestantes que lotaram a ...,1,1
4,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",banespa: saiba o que fazer durante a greve,NaN,"Agencia Estado, 09 Novembro 2000 | 13h32",A greve de funcionários do Banespa iniciada no...,1,1
5,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",tst: greve do banespa acaba amanhã,NaN,"Agencia Estado, 09 Novembro 2000 | 13h13","O presidente do Tribunal Superior do Trabalho,...",1,1
6,geral,economia,https://economia.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",estivadores estão em greve no porto de santos,NaN,"Agencia Estado, 29 Janeiro 2001 | 10h42","O Porto de Santos, o maior da América Latina, ...",1,1
7,geral,politica,https://politica.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",greve de caminhoneiros já afeta porto de santos,NaN,"Agencia Estado, 30 Janeiro 2001 | 17h45",A greve dos caminhoneiros iniciada ontem já es...,1,1
8,geral,brasil,"https://brasil.estadao.com.br/noticias/geral,g...","=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",greve: estivadores invadem armazém em santos,NaN,"Agencia Estado, 03 Abril 2001 | 05h00","Quatro estivadores, ainda não identificados pe...",1,1
9,geral,politica,https://politica.estadao.com.br/noticias/geral...,"=HIPERLINK(""D:\Dropbox\Projetos - Corbi\Acervo...",bancos abrem apesar da continuidade da greve n...,NaN,"Agencia Estado, 17 Julho 2001 | 11h01",As agências bancárias da capital baiana voltar...,1,1


In [24]:
df_news2 = pd.read_excel('df_news.xlsx',
                         sheet_name='raw_data')

# ao adicionar os nomes dos bancos subi de 336 para 408 observações! Criar uma lista com o nome dos bancos desde 2000
termos_greve  = ['greve','paralisação','interrupção','suspensão']
termos_banco  = ['bancários','banco']

bancos_publicos = ['banco do brasil', 'banco da amazônia', 'banco do nordeste', 
                    'banco de brasília','banco do estado', 'caixa econômica']
bancos_privados = ['abn','bmg','citibank','safra','itaú','santander','bradesco']
bancos_extintos = ['lavra','banco popular','banco real','santos',
                    'nossa caixa','hsbc','unibanco','banespa','banerj','banestado']

termos_banco = termos_banco + bancos_publicos + bancos_privados + bancos_extintos
termos_greve = '|'.join(termos_greve)
termos_banco = '|'.join(termos_banco)

df_news2.dropna(axis=0,how='all',subset=['manchete'],inplace=True)
df_news2['greve']  = df_news2['manchete'].str.contains(termos_greve, regex=True).astype(int)
df_news2['bancos'] = df_news2['manchete'].str.contains(termos_banco, regex=True).astype(int)
df_greve = df_news2[df_news2['greve']==1]
df_greve_bancos = df_news2[(df_news2['greve']==1) & (df_news2['bancos']==1)]
df_greve_bancos.reset_index(drop=True, inplace=True)

if not os.path.exists('bancos'):
    os.makedirs('bancos')

for file in df_greve_bancos['files']:
    shutil.copy2(file,os.getcwd()+'\\bancos')

#for index, file in enumerate(df_greve_bancos['files']):
#    df_greve_bancos.set_value(index, 'files', '=HIPERLINK("'+os.getcwd()+'\\bancos\\'+file+'";"'+file+'")')

writer = pd.ExcelWriter('df_news_banks.xlsx')
df_greve_bancos.to_excel(writer, 'banks_data')
writer.save()

### Definindo a categoria da greve

In [80]:
list_categ = ['policiais', 'bancários', 'metroviários', 'professores', 'bombeiros', 'petroleiros', 
              'caminhoneiros', 'metalúrgicos', 'servidores públicos', 'ônibus', 
              'rodoviários', 'auditores fiscais', 'comerciários', 'correios',
              'portuários','aeronautas', 'ferroviários']

categorias = []
for manchete in df_news['manchete']:
    
    x=False
    for elem in list_categ:
        if elem in manchete:
            categorias.append(elem)
            x=True
    
    if not x:
        categorias.append('nan')
        
        
df_news['greve_categoria'] = categorias

### Definindo a duração da greve

In [81]:
tempo = []
for manchete in df_news['manchete']:
    
    tempo_atual = re.findall('após (\d+) dias',manchete)
    tempo_atual = ''.join(tempo_atual)
    
    if tempo_atual != '':
        tempo.append(int(tempo_atual))
    else:
        tempo.append('nan')
        
df_news['greve_dias'] = tempo

### Definindo o local da greve

In [84]:
local = []
for conteudo in df_news['conteudo']:
    
    local_atual = re.findall('^(.{1,30}) \- ',unidecode(conteudo).lower())
    local_atual = ''.join(local_atual)
    
    if local_atual != '':
        local.append(local_atual)
    else:
        local.append('nan')
        
df_news['greve_local'] = local

In [86]:
df_news

,manchete,subtitulo,autor,conteudo,greve,greve_categoria,greve_dias,greve_local
7,policiais civis descumprem decisão judicial e ...,Eles alegam falta de recursos estruturais e de...,"Ricardo Araújo, Especial para O Estado 03 Jane...","NATAL - Correndo o risco de serem presos, os p...",1,policiais,nan,natal
19,"após 21 dias, policiais do rio grande do norte...",PMs ainda negociavam não punição; atrasados de...,"Ricardo Araújo, especial para o Estado 09 Jane...",NATAL - Policiais civis e militares do Rio Gra...,1,policiais,21,natal
21,"após 22 dias de paralisação, policiais retomam...",Patrulhamento e investigação são restabelecido...,"Ricardo Araújo, especial para o Estado 10 Jane...",NATAL - A garantia dada pelo governo do Rio Gr...,1,policiais,22,natal
24,metroviários de sp marcam greve para a próxima...,Funcionários distribuíram panfletos para passa...,"Renata Okumura, O Estado de S.Paulo 16 Janeiro...",SÃO PAULO – Os metroviários de São Paulo marca...,1,metroviários,nan,sao paulo
25,"no rio grande do norte, greves e protestos por...",Governo do Estado não pagou os vencimentos de ...,"Ricaro Araújo, especial para o Estado, O Estad...","No Rio Grande do Norte, o ano começou com grev...",1,nan,nan,nan
29,"em assembleia, sindicato de metroviários confi...","O protesto, segundo os sindicalistas, será con...",O Estado de S.Paulo 17 Janeiro 2018 | 21h13,Assembleia realizada pelo Sindicato dos Metrov...,1,metroviários,nan,nan
31,economista carioca faz greve de fome em solida...,"Richard Faulhaber, de 62 anos, divide espaço c...","Taís Seibt, ESPECIAL PARA O ESTADO, O Estado d...",PORTO ALEGRE - No hall de entrada da Assemblei...,1,nan,nan,porto alegre
32,governo de sp promete operar maior número de e...,Liminar determina a manutenção do efetivo de 8...,"Renata Okumura, O Estado de S.Paulo 17 Janeiro...",SÃO PAULO – Diante da possiblidade de greve do...,1,nan,nan,sao paulo
34,"greve dos metroviários prejudica 3,5 milhões; ...",Paralisação convocada por sindicato contra a c...,"Bruno Ribeiro, Juliana Diógenes e Marco Antôni...","SÃO PAULO - Ao menos 3,5 milhões de pessoas, s...",1,metroviários,nan,sao paulo
36,greve do metrô prejudica trânsito na capital p...,CET liberou o rodízio de veículos na cidade ne...,"Renata Okumura, O Estado de S.Paulo 18 Janeiro...",SÃO PAULO – A paralisação dos metroviários de ...,1,nan,nan,sao paulo
